In [1]:
import gymenv_v2
from gymenv_v2 import make_multiple_env
import numpy as np
from config import custom_config, easy_config, hard_config
from layers import Embedding
import tensorflow as tf
from policy import Policy
from rollout import rollout, rollout_envs
import os

import wandb
wandb.login()
run=wandb.init(project="finalproject", entity="orcs4529", tags=["training-easy"])

%load_ext autoreload
%autoreload 2

2022-12-12 17:05:42.665148: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-12 17:05:44.444292: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-12-12 17:05:44.444531: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [2]:
env = make_multiple_env(**easy_config) 

loading training instances, dir instances/train_10_n60_m60 idx 0
loading training instances, dir instances/train_10_n60_m60 idx 1
loading training instances, dir instances/train_10_n60_m60 idx 2
loading training instances, dir instances/train_10_n60_m60 idx 3
loading training instances, dir instances/train_10_n60_m60 idx 4
loading training instances, dir instances/train_10_n60_m60 idx 5
loading training instances, dir instances/train_10_n60_m60 idx 6
loading training instances, dir instances/train_10_n60_m60 idx 7
loading training instances, dir instances/train_10_n60_m60 idx 8
loading training instances, dir instances/train_10_n60_m60 idx 9


In [3]:
if not os.path.exists("results"):
    os.mkdir("results")

## Params

In [4]:
units = [64, 64]
activations = ['tanh', 'tanh']
lr = 0.01  ### CHANGE
num_episodes = 200
num_trajectories = 10 ### CHANGE
delta_std = 0.1
rollout_length = 1
time_limit = 50 ### CHANGE
gamma = 0.90

run_name = "easy_baseline2"

In [5]:
if not os.path.exists(f"results/{run_name}"):
    os.mkdir(f"results/{run_name}")

all_params = {
    "units": units, "activations": activations, "lr": lr, "num_episodes": num_episodes,
    "num_trajectories": num_trajectories, "delta_std": delta_std, "rollout_length": rollout_length,
    "time_limit": time_limit, "gamma": gamma
}
np.save(f"results/{run_name}/params", all_params)

## Running loop over episodes

In [ ]:
# initialize policy and test
policy = Policy(units, activations)
s = env.reset()
_ = policy.compute_prob(s)
rewards_record = []

for e in range(num_episodes):
    w_orig_cons, w_orig_cuts = policy.get_weights()
    epsilon_table_cons, epsilon_table_cuts = [], []
    train_rewards_table = []
    for t in range(num_trajectories):
        epsilon_cons = [np.random.randn(*x.shape)*delta_std for x in w_orig_cons]
        epsilon_cuts = [np.random.randn(*x.shape)*delta_std for x in w_orig_cuts]
        w_new_cons = [w_orig_cons[i] + epsilon_cons[i] for i in range(len(w_orig_cons))]
        w_new_cuts = [w_orig_cuts[i] + epsilon_cuts[i] for i in range(len(w_orig_cuts))]
        policy.set_weights(w_new_cons, w_new_cuts)
        rewards, times = rollout_envs(envs=env.envs, policy=policy, num_rollouts=1, rollout_length=rollout_length, gamma=gamma)
        epsilon_table_cons.append(epsilon_cons)
        epsilon_table_cuts.append(epsilon_cuts)
        # epsilon_table.append(epsilon)
        train_rewards_table.append(np.mean(rewards))
    
    train_rewards_table = np.array(train_rewards_table)
    train_rewards_table = (train_rewards_table - np.mean(train_rewards_table))/ (np.std(train_rewards_table) + 1e-8)

    grads_cons = []
    grads_cuts = []
    for j in range(len(w_orig_cons)):
        arr_cons = np.zeros(epsilon_table_cons[0][j].shape)
        arr_cuts = np.zeros(epsilon_table_cuts[0][j].shape)
        for i in range(len(epsilon_table_cons)):
            arr_cons += epsilon_table_cons[i][j] * train_rewards_table[i]
            arr_cuts += epsilon_table_cuts[i][j] * train_rewards_table[i]
        arr_cons /= (len(epsilon_table_cons) * delta_std)
        arr_cuts /= (len(epsilon_table_cuts) * delta_std)
        grads_cons.append(arr_cons)
        grads_cuts.append(arr_cuts)
    
    # assign back original weights and update
    w_cons = [w_orig_cons[i] - lr*grads_cons[i] for i in range(len(w_orig_cons))]
    w_cuts = [w_orig_cuts[i] - lr*grads_cuts[i] for i in range(len(w_orig_cuts))]

    policy.set_weights(w_cons, w_cuts)
    
    # evaluate rewards
    eval_r, _ = rollout_envs(envs=env.envs, policy=policy, num_rollouts=1, rollout_length=time_limit, gamma=gamma)
    print(f"Episode {e}:")
    print('mean',np.mean(eval_r),'max',np.max(eval_r),'min',np.min(eval_r),'std',np.std(eval_r))
    print("")
    rewards_record.append(np.mean(eval_r))
    
    fixedWindow = 100
    movingAverage = 0
    if len(rewards_record) >= fixedWindow:
        movingAverage = np.mean(rewards_record[len(rewards_record)-fixedWindow:len(rewards_record)-1])
        
    wandb.log({"Training reward" : float(rewards_record[-1]), "Training reward moving average": movingAverage})
    np.save(f"results/{run_name}/reward{e}", eval_r)

Restricted license - for non-production use only - expires 2024-10-28


2022-12-12 17:06:05.505348: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-12-12 17:06:05.505621: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-12 17:06:05.505708: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (learn2cut): /proc/driver/nvidia/version does not exist
2022-12-12 17:06:05.506963: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the approp